### AQC
AQC的核心思想，绝热量子演化: H(f(s)) = (1 − f(s))H0 + f(s)H1, 0 ≤ s ≤ 1.
H1 是给定的
这里H0 是一个用来实现初态制备的，
$$
H_0=\sigma_x \otimes Q_b=\left(\begin{array}{cc}
0 & Q_b \\
Q_b & 0
\end{array}\right)
$$
$Q_b=$ $I_N-|b\rangle\langle b|$. 
then $H_0$ is a Hermitian matrix and the null space of $H_0$ is $\operatorname{Null}\left(H_0\right)=\operatorname{span}\{|\widetilde{b}\rangle,|\bar{b}\rangle\}$. Here, $|\widetilde{b}\rangle=$ $|0, b\rangle:=(b, 0)^{\top},|\bar{b}\rangle=|1, b\rangle:=(0, b)^{\top}$. The dimension of $H_0$ is $2 N$ and one ancilla qubit is needed to enlarge the matrix block. We also define
$$
H_1=\sigma_{+} \otimes\left(A Q_b\right)+\sigma_{-} \otimes\left(Q_b A\right)=\left(\begin{array}{cc}
0 & A Q_b \\
Q_b A & 0
\end{array}\right) .
$$
Here, $\sigma_{ \pm}=\frac{1}{2}\left(\sigma_x \pm \mathrm{i} \sigma_y\right)$. Note that if $|x\rangle$ satisfies $A|x\rangle \propto|b\rangle$, we have $Q_b A|x\rangle=Q_b|b\rangle=0$. Then $\operatorname{Null}\left(H_1\right)=\operatorname{span}\{|\widetilde{x}\rangle,|\bar{b}\rangle\}$ with $|\widetilde{x}\rangle=|0, x\rangle$. Since $Q_b$ is a projection operator, the gap between 0 and the rest of the eigenvalues of $H_0$ is 1 . The gap between 0 and the rest of the eigenvalues of $H_1$ is bounded from below by $1 / \kappa$ (see Appendix A).
### GATE-BASED IMPLEMENTATION OF AQC
We briefly discuss how to implement $\mathrm{AQC}(\mathrm{p})$ and $\mathrm{AQC}(\mathrm{exp})$ on a gate-based quantum computer. Since $\left|\psi_T(s)\right\rangle=\mathcal{T} \exp \left(-\mathrm{i} T \int_0^s H\left(f\left(s^{\prime}\right)\right) d s^{\prime}\right)\left|\psi_T(0)\right\rangle$, where $\mathcal{T}$ is the time-ordering operator, it is sufficient to implement an efficient time-dependent Hamiltonian simulation of $H(f(s))$.

One straightforward approach is to use the Trotter splitting method. The lowest order approximation takes the form
$$
\begin{aligned}
& \mathcal{T} \exp \left(-\mathrm{i} T \int_0^s H\left(f\left(s^{\prime}\right)\right) \mathrm{d} s^{\prime}\right) \approx \prod_{m=1}^M \exp \left(-\mathrm{i} T h H\left(f\left(s_m\right)\right)\right) \\
& \approx \prod_{m=1}^M \exp \left(-\mathrm{i} T h\left(1-f\left(s_m\right)\right) H_0\right) \exp \left(-\mathrm{i} T h f\left(s_m\right) H_1\right),
\end{aligned}
$$
where $h=s / M, s_m=m h$. It is proved in [31] that the error of such an approximation is $O$ (poly $\left.(\log (N)) T^2 / M\right)$, which indicates that to achieve an $\epsilon$-approximation, it suffices to choose $M=$ $O\left(\operatorname{poly}(\log (N)) T^2 / \epsilon\right)$. 


In [3]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import numpy as np
from qiskit import QuantumCircuit
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import numpy as np
from qiskit import QuantumCircuit


def gray_code(b):
    '''Gray code of b.
    Args:
        b: int:
            binary integer
    Returns:
        Gray code of b.
    '''
    return b ^ (b >> 1)


def gray_permutation(a):
    '''Permute the vector a from binary to Gray code order.

    Args:
        a: vector
            1D NumPy array of size 2**n
    Returns:
        vector:
            Gray code permutation of a
    '''
    b = np.zeros(a.shape[0])
    for i in range(a.shape[0]):
        b[i] = a[gray_code(i)]
    return b


def sfwht(a):
    '''Scaled Fast Walsh-Hadamard transform of input vector a.

    Args:
        a: vector
            1D NumPy array of size 2**n.
    Returns:
        vector:
            Scaled Walsh-Hadamard transform of a.
    '''
    n = int(np.log2(a.shape[0]))
    for h in range(n):
        for i in range(0, a.shape[0], 2**(h+1)):
            for j in range(i, i+2**h):
                x = a[j]
                y = a[j + 2**h]
                a[j] = (x + y) / 2
                a[j + 2**h] = (x - y) / 2
    return a


def compute_control(i, n):
    '''Compute the control qubit index based on the index i and size n.'''
    if i == 4**n:
        return 1
    return 2*n - int(np.log2(gray_code(i-1) ^ gray_code(i)))


def compressed_uniform_rotation(a, ry=True):
    '''Compute a compressed uniform rotation circuit based on the thresholded
    vector a.

    Args:
        a: vector:
            A thresholded vector a a of dimension 2**n
        ry: bool
            uniform ry rotation if true, else uniform rz rotation
    Returns:
        circuit
            A qiskit circuit representing the compressed uniform rotation.
    '''
    n = int(np.log2(a.shape[0])/2)
    circ = QuantumCircuit(2*n + 1)

    i = 0
    while i < a.shape[0]:
        parity_check = 0

        # add the rotation gate
        if a[i] != 0:
            if ry:
                circ.ry(a[i], 0)
            else:
                circ.rz(a[i], 0)

        # loop over sequence of consecutive zeros
        while True:
            ctrl = compute_control(i+1, n)
            # toggle control bit
            parity_check = (parity_check ^ (1 << (ctrl-1)))
            i += 1
            if i >= a.shape[0] or a[i] != 0:
                break

        # add CNOT gates
        for j in range(1, 2*n+1):
            if parity_check & (1 << (j-1)):
                circ.cx(j, 0)

    return circ

def fable(a, epsilon=None):
    '''FABLE - Fast Approximate BLock Encodings.

    Args:
        a: array
            matrix to be block encoded.
        epsilon: float >= 0
            (optional) compression threshold.
    Returns:
        circuit: qiskit circuit
            circuit that block encodes A
        alpha: float
            subnormalization factor
    '''
    epsm = np.finfo(a.dtype).eps
    alpha = np.linalg.norm(np.ravel(a), np.inf)
    if alpha > 1:
        alpha = alpha + np.sqrt(epsm)
        a = a/alpha
    else:
        alpha = 1.0

    n, m = a.shape
    if n != m:
        k = max(n, m)
        a = np.pad(a, ((0, k - n), (0, k - m)))
        n = k
    logn = int(np.ceil(np.log2(n)))
    if n < 2**logn:
        a = np.pad(a, ((0, 2**logn - n), (0, 2**logn - n)))
        n = 2**logn

    a = np.ravel(a)

    if all(np.abs(np.imag(a)) < epsm):  # real data
        a = gray_permutation(
                sfwht(
                    2.0 * np.arccos(np.real(a))
                )
            )
        # threshold the vector
        if epsilon:
            a[abs(a) <= epsilon] = 0
        # compute circuit
        OA = compressed_uniform_rotation(a)
    else:  # complex data
        # magnitude
        a_m = gray_permutation(
                sfwht(
                    2.0 * np.arccos(np.abs(a))
                )
            )
        if epsilon:
            a_m[abs(a_m) <= epsilon] = 0

        # phase
        a_p = gray_permutation(
                sfwht(
                    -2.0 * np.angle(a)
                )
            )
        if epsilon:
            a_p[abs(a_p) <= epsilon] = 0

        # compute circuit
        OA = compressed_uniform_rotation(a_m).compose(
                compressed_uniform_rotation(a_p, ry=False)
            )

    circ = QuantumCircuit(2*logn + 1)

    # diffusion on row indices
    for i in range(logn):
        circ.h(i+1)

    # matrix oracle
    circ = circ.compose(OA)

    # swap register
    for i in range(logn):
        circ.swap(i+1,  i+logn+1)

    # diffusion on row indices
    for i in range(logn):
        circ.h(i+1)

    # reverse bits because of little-endiannes
    circ = circ.reverse_bits()

    return circ, alpha

In [115]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Operator
from scipy.linalg import expm

def setting(A,b,T,M):
    dimA = A.shape[0]
    n = 2*int(np.log2(dimA))
    N = 2*2**n
    an = 2*n +1
    b = b / np.linalg.norm(b) 
    d = dimA
    return N, an,n, b, d, T, M
    
def get_H0(b):
    Q_b = np.eye(b.shape[0]) - np.outer(b, b.conj())
    H0 = np.kron(np.array([[0, 1], [1, 0]]), Q_b)
    return H0

def get_H1(A, b):
    Q_b = np.eye(b.shape[0]) - np.outer(b, b.conj())
    H1 = np.kron(np.array([[0, 1], [0, 0]]), A @ Q_b) + np.kron(np.array([[0, 0], [1, 0]]), Q_b @ A)
    return H1

# Define the initial state |psi_T(0)>
def initial_state(an,b,n):
    qc = QuantumCircuit(an,an)  # N system qubits + 1 ancilla
    qc.initialize(b, 0)
    return qc

# Implement the Trotterized evolution
def discrete_step(qc, H0, H1, f,n,M,T):
    U1 = expm(-1j * (1 - f) * H0*f/M*T )
    U2 = expm(- 1j * f * H1 *f/M*T)
    qc.unitary(U1, range(n))
    qc.unitary(U2, range(n))
# Main function to simulate AQC
def simulate_AQC(A,b,T,M,f=lambda s: s):
    N, an,n, b, d, T, M = setting(A,b,T,M)
    H0 = get_H0(b)
    H1 = get_H1(A,b)
    qc = initial_state(n,b,n)
    
    for m in range(M):
        s = (m + 1) /M
        f_s = f(s)
        discrete_step(qc, H0, H1, f_s, n,M,T)
    return qc
    
A = np.array([[2, 1], [1, 0]])
x = np.array([1, 1])
b = A @ x

final_qc = simulate_AQC(A,b,800,200)

In [10]:
import qiskit
circuit = qiskit.QuantumCircuit(2, 2)
circuit.cx(0, 1)
## get unitary
unitary = qiskit.quantum_info.Operator(circuit).data
print(unitary)

[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]]


In [116]:
final_qc.draw()

┌─────────────────────────────┐┌──────────┐┌──────────┐┌──────────┐»
q_0: ┤ Initialize(0.94868,0.31623) ├┤0         ├┤0         ├┤0         ├»
     └─────────────────────────────┘│  Unitary ││  Unitary ││  Unitary │»
q_1: ───────────────────────────────┤1         ├┤1         ├┤1         ├»
                                    └──────────┘└──────────┘└──────────┘»
c: 2/═══════════════════════════════════════════════════════════════════»
                                                                        »
«     ┌──────────┐┌──────────┐┌──────────┐┌──────────┐┌──────────┐┌──────────┐»
«q_0: ┤0         ├┤0         ├┤0         ├┤0         ├┤0         ├┤0         ├»
«     │  Unitary ││  Unitary ││  Unitary ││  Unitary ││  Unitary ││  Unitary │»
«q_1: ┤1         ├┤1         ├┤1         ├┤1         ├┤1         ├┤1         ├»
«     └──────────┘└──────────┘└──────────┘└──────────┘└──────────┘└──────────┘»
«c: 2/════════════════════════════════════════════════════════════════════════»
«                                                                             »
«     ┌──────────┐┌──────────┐┌──────────┐┌──────────┐┌──────────┐┌──────────┐»
«q_0: ┤0         ├┤0         ├┤0         ├┤0         ├┤0         ├┤0         ├»
«     │  Unitary ││  Unitary ││  Unitary ││  Unitary ││  Unitary ││  Unitary │»
«q_1: ┤1         ├┤1         ├┤1         ├┤1         ├┤1         ├┤1         ├»
«     └──────────┘└──────────┘└──────────┘└──────────┘└──────────┘└──────────┘»
«c: 2/════════════════════════════════════════════════════════════════════════»
«                                                                             »
«     ┌──────────┐┌──────────┐┌──────────┐┌──────────┐┌──────────┐┌──────────┐»
«q_0: ┤0         ├┤0         ├┤0         ├┤0         ├┤0         ├┤0         ├»
«     │  Unitary ││  Unitary ││  Unitary ││  Unitary ││  Unitary ││  Unitary │»
«q_1: ┤1         ├┤1         ├┤1         ├┤1         ├┤1         ├┤1         ├»
«     └──────────┘└──────────┘└──────────┘└──────────┘└──────────┘└──────────┘»
«c: 2/════════════════════════════════════════════════════════════════════════»
«                                                                             »
«     ┌──────────┐┌──────────┐┌──────────┐┌──────────┐┌──────────┐┌──────────┐»
«q_0: ┤0         ├┤0         ├┤0         ├┤0         ├┤0         ├┤0         ├»
«     │  Unitary ││  Unitary ││  Unitary ││  Unitary ││  Unitary ││  Unitary │»
«q_1: ┤1         ├┤1         ├┤1         ├┤1         ├┤1         ├┤1         ├»
«     └──────────┘└──────────┘└──────────┘└──────────┘└──────────┘└──────────┘»
«c: 2/════════════════════════════════════════════════════════════════════════»
«                                                                             »
«     ┌──────────┐┌──────────┐┌──────────┐┌──────────┐┌──────────┐┌──────────┐»
«q_0: ┤0         ├┤0         ├┤0         ├┤0         ├┤0         ├┤0         ├»
«     │  Unitary ││  Unitary ││  Unitary ││  Unitary ││  Unitary ││  Unitary │»
«q_1: ┤1         ├┤1         ├┤1         ├┤1         ├┤1         ├┤1         ├»
«     └──────────┘└──────────┘└──────────┘└──────────┘└──────────┘└──────────┘»
«c: 2/════════════════════════════════════════════════════════════════════════»
«                                                                             »
«     ┌──────────┐┌──────────┐┌──────────┐┌──────────┐┌──────────┐┌──────────┐»
«q_0: ┤0         ├┤0         ├┤0         ├┤0         ├┤0         ├┤0         ├»
«     │  Unitary ││  Unitary ││  Unitary ││  Unitary ││  Unitary ││  Unitary │»
«q_1: ┤1         ├┤1         ├┤1         ├┤1         ├┤1         ├┤1         ├»
«     └──────────┘└──────────┘└──────────┘└──────────┘└──────────┘└──────────┘»
«c: 2/════════════════════════════════════════════════════════════════════════»
«                                                                             »
«     ┌──────────┐┌──────────┐┌──────────┐┌──────────┐┌──────────┐┌──────────┐»
«q_0: ┤0         ├┤0         ├┤0         ├┤0   

In [117]:
## excute the circuit and get the statevector
from qiskit import QuantumCircuit, execute, Aer

# execute the circuit on a simulator
backend = Aer.get_backend('statevector_simulator')
job = execute(final_qc, backend)
result = job.result()

# get the statevector
np.set_printoptions(precision=3, suppress=True)
np.set_printoptions(formatter={'float': '{: 0.3f}'.format})
statevector = result.get_statevector()
print(statevector)


Statevector([-0.71 +0.j   , -0.704+0.j   ,  0.   -0.001j,  0.   +0.003j],
            dims=(2, 2))


In [118]:
x

array([1, 1])

In [119]:
psi_T = [statevector.data[0],statevector.data[1]]
psi_T = psi_T / np.linalg.norm(psi_T)
overlap = np.abs(np.vdot(psi_T,x/np.linalg.norm(x))) ** 2
print("Overlap with target state: ", overlap)

Overlap with target state:  0.9999814761931947


## 全是unitary 的测试版本

In [4]:
import numpy as np
import pennylane as qml
from scipy.linalg import expm
def setting(A,b,T,M):
    dimA = A.shape[0]
    n = 2*int(np.log2(dimA))
    N = 2*2**n
    an = 2*n +1
    b = b / np.linalg.norm(b) 
    d = dimA
    return N, an,n, b, d, T, M

def get_H0(b):
    Q_b = np.eye(b.shape[0]) - np.outer(b, b.conj())
    H0 = np.kron(np.array([[0, 1], [1, 0]]), Q_b)
    return H0

def get_H1(A,b):
    Q_b = np.eye(b.shape[0]) - np.outer(b, b.conj())
    H1 = np.kron(np.array([[0, 1], [0, 0]]), A @ Q_b) + np.kron(np.array([[0, 0], [1, 0]]), Q_b @ A)
    return H1

# Define the initial state |psi_T(0)>
def initial_state(b,n):
    qml.QubitStateVector(b, wires=range(int(n/2),n))

# Implement the Trotterized evolution
def discrete_step(H0, H1, f,n,M,T):
    U1 = expm(-1j * (1 -f) * H0*T/M - 1j * f *H1 *T/M)
    qml.QubitUnitary(U1, range(n))

@qml.qnode(qml.device("default.qubit", wires=2))
def simulate_AQC(A,b,T,M,f=lambda s: s):
    dimA = A.shape[0]
    n = 2*int(np.log2(dimA))
    b = b / np.linalg.norm(b) 
    H0 = get_H0(b)
    H1 = get_H1(A,b)
    initial_state(b,n)
    for m in range(M):
        s = (m + 1) /M
        f_s = f(s)
        discrete_step(H0, H1, f_s, n,M,T)
    return qml.state()
# Main function to simulate AQC
A = np.array([[2, 1], [1, 0]])
x = np.array([2,2])
b = A @ x
psi_T = simulate_AQC(A, b,1000,200)
psi_T = psi_T/np.linalg.norm(psi_T)
fidelity = np.abs(np.vdot(psi_T[:2],x/np.linalg.norm(x))) ** 2

In [12]:
## print circuit of pennylane
print(qml.draw(simulate_AQC)(A, b,1000,200))


0: ──────╭U(M0)─╭U(M1)─╭U(M2)─╭U(M3)─╭U(M4)─╭U(M5)─╭U(M6)─╭U(M7)─╭U(M8)─╭U(M9)─╭U(M10)─╭U(M11)
1: ──|Ψ⟩─╰U(M0)─╰U(M1)─╰U(M2)─╰U(M3)─╰U(M4)─╰U(M5)─╰U(M6)─╰U(M7)─╰U(M8)─╰U(M9)─╰U(M10)─╰U(M11)

──╭U(M12)─╭U(M13)─╭U(M14)─╭U(M15)─╭U(M16)─╭U(M17)─╭U(M18)─╭U(M19)─╭U(M20)─╭U(M21)─╭U(M22)─╭U(M23)
──╰U(M12)─╰U(M13)─╰U(M14)─╰U(M15)─╰U(M16)─╰U(M17)─╰U(M18)─╰U(M19)─╰U(M20)─╰U(M21)─╰U(M22)─╰U(M23)

──╭U(M24)─╭U(M25)─╭U(M26)─╭U(M27)─╭U(M28)─╭U(M29)─╭U(M30)─╭U(M31)─╭U(M32)─╭U(M33)─╭U(M34)─╭U(M35)
──╰U(M24)─╰U(M25)─╰U(M26)─╰U(M27)─╰U(M28)─╰U(M29)─╰U(M30)─╰U(M31)─╰U(M32)─╰U(M33)─╰U(M34)─╰U(M35)

──╭U(M36)─╭U(M37)─╭U(M38)─╭U(M39)─╭U(M40)─╭U(M41)─╭U(M42)─╭U(M43)─╭U(M44)─╭U(M45)─╭U(M46)─╭U(M47)
──╰U(M36)─╰U(M37)─╰U(M38)─╰U(M39)─╰U(M40)─╰U(M41)─╰U(M42)─╰U(M43)─╰U(M44)─╰U(M45)─╰U(M46)─╰U(M47)

──╭U(M48)─╭U(M49)─╭U(M50)─╭U(M51)─╭U(M52)─╭U(M53)─╭U(M54)─╭U(M55)─╭U(M56)─╭U(M57)─╭U(M58)─╭U(M59)
──╰U(M48)─╰U(M49)─╰U(M50)─╰U(M51)─╰U(M52)─╰U(M53)─╰U(M54)─╰U(M55)─╰U(M56)─╰U(M57)─╰U(M58)─╰U(M59)

──╭U(M60)─╭U(M61)─╭U(

In [5]:
psi_T[:2],b

(tensor([-0.70917167+0.j, -0.70503336+0.j], requires_grad=True), array([6, 2]))

In [6]:
fidelity

0.9999879420849797

In [7]:
psi_T

tensor([-0.70917167+0.j        , -0.70503336+0.j        ,
         0.        +0.00059119j,  0.        -0.00177358j], requires_grad=True)

## 只用了numpy库，没有用到其他库

In [24]:
import numpy as np
from scipy.linalg import expm
# Define Hamiltonians H0 and H1
def get_H0(b):
    Q_b = np.eye(b.shape[0]) - np.outer(b, b.conj())
    H0 = np.kron(np.array([[0, 1], [1, 0]]), Q_b)
    return H0

def get_H1(A, b):
    Q_b = np.eye(b.shape[0]) - np.outer(b, b.conj())
    H1 = np.kron(np.array([[0, 1], [0, 0]]), A @ Q_b) + np.kron(np.array([[0, 0], [1, 0]]), Q_b @ A)
    return H1

# Define the initial state |psi_T(0)>
def initial_state(b):
    state = np.kron(np.array([1, 0]), b)
    return state
def discrete_step(state, H0, H1, f, T, M):
    DM = 2
    for _ in range(DM):
        U0 = np.eye(H0.shape[0]) - 1j *(1-f) * H0 * T/M/DM
        U1 = np.eye(H1.shape[0]) - 1j * f * H1 * T/M/DM
        U = U0 @ U1
        state = U @ state 
        state = state / np.linalg.norm(state)
    return state

def simulate_AQC(A, b, T,M):
    H0 = get_H0( b)
    H1 = get_H1(A, b)
    state = initial_state(b)
    print(state)
    for m in range(M):
        s = (m + 1) /M
        state = discrete_step(state,H0, H1, s, T, M)
        state = state / np.linalg.norm(state)
    return state
# Define parameters
A = np.array([[1, 1], [1, 0]])
x = np.array([2,1])
b = A @ x
psi_T = simulate_AQC(A, b,2,200)
psi_T =psi_T[:2]/np.linalg.norm(psi_T[:2])
# Compute the overlap with the target state |psi_T>
fidelity = np.abs(np.vdot(psi_T[:2],x/np.linalg.norm(x))) ** 2
fidelity


[3 2 0 0]


0.9941004790923013

In [23]:
psi_T,b

(array([0.85743521+0.j, 0.51459194+0.j]), array([3, 2]))

In [ ]:
psi_T,b